In [24]:
pip install textwrap

^C
Note: you may need to restart the kernel to use updated packages.


In [27]:
import pandas as pd
import textwrap

excel_file = 'C:/Z/Book1.csv'
df = pd.read_csv(excel_file)

def split_text(text, max_length):
    if len(text) <= max_length:
        return text, ""
    
    split_index = text.rfind(' ', 0, max_length)
    if split_index == -1:
        split_index = max_length
    
    return text[:split_index], text[split_index+1:]

max_length = 35
df['NAME1'], df['NAME2'] = zip(*df['NAME1'].apply(lambda x: split_text(x, max_length)))

# Concatenate NAME3, NAME4, and STREET
df['CONCATENATED'] = df['NAME3'] + df['NAME4'] + df['STREET']

# Handle NaN or non-string values in 'CONCATENATED' gracefully
def wrap_text(x):
    if isinstance(x, str):
        return '\n'.join(textwrap.wrap(x, width=35, break_long_words=False, replace_whitespace=False))
    else:
        return ""

df['CONCATENATED'] = df['CONCATENATED'].apply(wrap_text)
wrapped_text = df['CONCATENATED']

# Split the concatenated value into three columns of 35 characters each without word breaks
df[['NAME3', 'NAME4', 'STREET']] = wrapped_text.str.split('\n', expand=True, n=2)

# Drop the CONCATENATED column if you no longer need it
df.drop(columns=['CONCATENATED'], inplace=True)

df['BANKN'] = df['BANKN'].apply(lambda x: str(x).replace("'", "").strip() if pd.notna(x) else "123")
df['BANKN'].fillna('123', inplace=True)

# Mapping for 'J_1IPANNO' values to 'TITLE_MEDI'
title_mapping = {
    'C': 'Company',
    'P': 'Individual or Proprietor',
    'H': 'HUF',
    'A': 'Association of Persons',
    'B': 'Body of Individuals',
    'G': 'Government Agency',
    'J': 'Artificial Juridical Person',
    'L': 'Local Authority',
    'F': 'Firm',
    'T': 'Trust'
}

# Set 'Government' for blank values in 'J_1IPANNO'
df['TITLE_MEDI'] = df['J_1IPANNO'].str[3].map(title_mapping)
df['TITLE_MEDI'].fillna('Government', inplace=True)

# Corrected the condition for 'AKONT' column based on the 'Category' column
df['AKONT'] = df.apply(lambda row: 3000000 if row['Category'] == 'Consumables' else 3020000, axis=1)

# Handle the 'MSME Uam No' column gracefully with a try-except block
def split_msme_uam(msme_uam):
    try:
        msme_uam = str(msme_uam).strip()  # Convert to string and strip whitespace
        if len(msme_uam) >= 15:
            return f"{msme_uam[:5]}-{msme_uam[5:7]}-{msme_uam[7:9]}-{msme_uam[9:]}"
        else:
            return ""  # Return an empty string for values with insufficient length
    except Exception as e:
        return ""  # Return an empty string for non-string values

df['MSME Uam No'] = df['MSME Uam No'].apply(split_msme_uam)

# Specify the ExcelWriter with the openpyxl engine
excel_writer = pd.ExcelWriter('C:/Z/output_excel_file.xlsx', engine='openpyxl')

# Convert the DataFrame to Excel with adjusted column width
df.to_excel(excel_writer, index=False)

# Get the xlsxwriter workbook and worksheet objects
workbook = excel_writer.book
worksheet = excel_writer.sheets['Sheet1']

# Iterate through all columns and set the width to fit the contents
for column in worksheet.columns:
    max_length = 0
    column_name = column[0].column_letter
    for cell in column:
        try:  # Necessary to avoid error on empty cells
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2)
    worksheet.column_dimensions[column_name].width = adjusted_width

# Save the Excel file
excel_writer.save()


C:\Users\Varun.Herlekar\AppData\Local\Temp\ipykernel_19244\2106220245.py:100: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
